In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# fixed parameters
hyper_epoch = 50
threshold = -1.3
epochs = 50
batch_size = 32
clip = 0.4
lr = 1e-4
epochs_type = epochs

In [ ]:
for _ in range(hyper_epoch):
    # model parameters

    head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update,SimplyInteraction],p=[0.4,0.2,0.4])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_atomInfo_otherInfo.pickle',\
                             '../Data/{}_data_ACSF_SOAP_atomInfo_otherInfo.pickle'])
    logLoss = np.random.choice([True,False])
    weight = np.random.rand()*3 if logLoss else np.random.rand()*0.8
    dim = int(np.random.choice([256,512]))
    interleave = np.random.choice([True,False])
    BatchNorm = np.random.choice([True,False],p=[0.6,0.4])
    if interleave:
        if dim == 768:
            layer1 = int(np.random.choice([3,4]))
            layer2 = layer1
        else:
            layer1 = int(np.random.choice([3,4,5]))
            layer2 = layer1            
    else:
        if dim == 768:        
            layer1 = int(np.random.choice([3,4]))
            layer2 = int(np.random.choice([3,4]))
        else:
            layer1 = int(np.random.choice([3,4,5]))
            layer2 = int(np.random.choice([3,4,5]))
            
    if dim == 768:
        factor = int(np.random.choice([2,3]))
    else:
        factor = int(np.random.choice([2,3,4]))
        
    useMax = np.random.choice([True,False])
    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [head,data,dim,logLoss,weight,layer1,layer2,factor,\
                                                  BatchNorm,useMax,interleave]])))

    train_dl,val_dl = get_data(data,batch_size)
    model = GNN_MataLayer(head,head_mol2,head_atom,head_edge,\
                          dim,layer1,layer2,factor,**data_dict[data],\
                          BatchNorm=BatchNorm,useMax=useMax,interleave=interleave).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    
    model,train_loss_perType,val_loss_perType,bestWeight = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,\
                                                                    scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
    if model is not None:
        save_results2(train_loss_perType,val_loss_perType,head,data,dim,logLoss,weight,layer1,layer2,factor,\
                        BatchNorm,useMax,interleave)